In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")

In [21]:
url="\dataset\TANKTEMP\TANKTEMP060-092.txt"

# date_date=lambda x: pd.datetime.strptime(x )
# data=pd.read_csv(url, delim_whitespace=True, names=('date_julian','value'), dtype={'data_julian': str, 'value': np.float64})
# data=pd.read_csv(url, delim_whitespace=True, names=('date_julian','value'))
# data=pd.read_csv(url,delim_whitespace=True, header=None)
data=pd.read_excel(url)

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [108]:
import calendar

def JulianDate_to_MMDDYYY(y,jd):
    month = 1
    day = 0
    while jd - calendar.monthrange(y,month)[1] > 0 and month <= 12:
        jd = jd - calendar.monthrange(y,month)[1]
        month = month + 1
    print (month,jd,y)
    print (type(month))
    

JulianDate_to_MMDDYYY(2008,67)



3 7 2008
<class 'int'>


In [73]:
calendar.monthrange(2019,1)[1]

31

In [27]:

url="/dataset/TANKTEMP/TANKTEMP060-092.txt"

data = open('TANKTEMP060-092.txt', "r")

In [29]:
print(data)

<_io.TextIOWrapper name='TANKTEMP060-092.txt' mode='r' encoding='cp874'>


In [47]:
data = pd.read_csv('TANKTEMP060-092.txt',delimiter="\t", header=None, names=["date", "value"])

In [49]:
data.head()

,date,value
0,2015-085-01:47:10.114,19.311741
1,2015-085-01:47:10.882,19.311700
2,2015-085-01:48:22.882,19.311700
3,2015-085-01:49:18.114,19.311741
4,2015-085-01:50:46.882,19.311700


In [65]:
data['da']=data.value*2

In [66]:
data

,date,value,da
0,2015-085-01:47:10.114,19.311741,38.623482
1,2015-085-01:47:10.882,19.311700,38.623400
2,2015-085-01:48:22.882,19.311700,38.623400
3,2015-085-01:49:18.114,19.311741,38.623482
4,2015-085-01:50:46.882,19.311700,38.623400
...,...,...,...
6074,2015-092-02:40:30.114,20.537634,41.075268
6075,2015-092-02:42:38.114,20.537634,41.075268
6076,2015-092-02:44:46.114,20.537634,41.075268
6077,2015-092-02:46:54.114,20.591398,41.182796


In [105]:
d=data['date'][0]
dd = d.split('-')
print(type(dd[1]))
print(len(dd))



<class 'str'>
3


In [141]:
def JulianToStantard(yy, jd):
    month = 1
    day= 0
    while jd - calendar.monthrange(yy,month)[1] > 0 and month <= 12:
        std = jd - calendar.monthrange(yy,month)[1]
        month = month + 1
    return (yy,month,std)

# for i in range(len(data)):
for i in range(2):
    dd=data['date'][i].split('-')
    yy=int(dd[0])
    jd=int(dd[1])
    print(y,jd)
#     ss=int(dd[2])
    JulianToStantard(yy,jd)
#     data['ddd'][i]= 9

    

# data
#     for j in range(len(dd)):
        
#     print(dd[1])

2015 85


IllegalMonthError: bad month number 13; must be 1-12

In [134]:
calendar.monthrange(2015,12)[1] 

31

In [113]:
import datetime

x = datetime.datetime(2020, 5, 17, 4,30,21)

print(x)

2020-05-17 04:30:21


In [242]:
a = "2016-03-22 12:33:45.7565"
datetime.datetime.strptime(a, "%Y-%m-%d %H:%M:%S.%f")

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [241]:
print (datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])

2020-03-02 11:51:45.654


In [157]:

def JulianDate_to_MMDDYYY(y,jd):
    month = 1
    day = 0
    while jd - calendar.monthrange(y,month)[1] > 0 and month <= 12:
#         print(calendar.monthrange(y,month) )
        jd = jd - calendar.monthrange(y,month)[1]
        month = month + 1
        print (month,jd,y)
    print (month,jd,y)
for i in range(2):
    dd=data['date'][i].split('-')
    yy=int(dd[0])
    jd=int(dd[1])
    print(y,jd)
    JulianDate_to_MMDDYYY(yy,jd)

2015 85
2 54 2015
3 26 2015
3 26 2015
2015 85
2 54 2015
3 26 2015
3 26 2015


In [195]:
import calendar

def JulianDate_to_MMDDYYY(y,jd):
    month = 1
    day = 0
    while jd - calendar.monthrange(y,month)[1] > 0 and month <= 12:
        jd = jd - calendar.monthrange(y,month)[1]
        month = month + 1
#     print (month,jd,y)


for i in range(len(data)):
    dd=data['date'][i].split('-')
    yy=int(dd[0])
    jd=int(dd[1])
    JulianDate_to_MMDDYYY(yy,jd)
#     data['ddd'][i]=jd
#     data['ddd'][i]=jd
# data

In [193]:
import calendar


for i in range(len(data)):
    dd=data['date'][i].split('-')
    yy=int(dd[0])
    jd=int(dd[1])
    
    month = 1
    day = 0
    while jd - calendar.monthrange(y,month)[1] > 0 and month <= 12:
        jd = jd - calendar.monthrange(y,month)[1]
        month = month + 1
    data['ddd'][i]=jd
    

#     data['ddd'][i]=jd

In [194]:
data

,date,value,da,ddd
0,2015-085-01:47:10.114,19.311741,38.623482,26
1,2015-085-01:47:10.882,19.311700,38.623400,26
2,2015-085-01:48:22.882,19.311700,38.623400,26
3,2015-085-01:49:18.114,19.311741,38.623482,26
4,2015-085-01:50:46.882,19.311700,38.623400,26
...,...,...,...,...
6074,2015-092-02:40:30.114,20.537634,41.075268,2
6075,2015-092-02:42:38.114,20.537634,41.075268,2
6076,2015-092-02:44:46.114,20.537634,41.075268,2
6077,2015-092-02:46:54.114,20.591398,41.182796,2


In [259]:
import datetime

x = datetime.datetime(2020, 5, 17, 4,30,28)
print(x)

2020-05-17 04:30:28


In [316]:
sec='2:54:45.345'
sec1=sec.split(':')
sec1[1]
print(type(sec1[2]))
print(sec1[2][:-])
mm=sec1[2][-3:]
print(mm)
print(type(mm))
# x = datetime.datetime(2020, 5, 17, int(sec1[0]),int(sec1[1]),int(sec1[2]),int(mm))
# print(x)

# print(datetime.strptime('30/03/09 16:31:32.123', '%d/%m/%y %H:%M:%S.%f'))

<class 'str'>
45.345
345
<class 'str'>


In [312]:
import calendar
import datetime

for i in range(len(data)):
    dd=data['date'][i].split('-')
    yy=int(dd[0])
    jd=int(dd[1])
    sec=dd[2].split(':')
    
    month = 1
    day = 0
    while jd - calendar.monthrange(y,month)[1] > 0 and month <= 12:
        jd = jd - calendar.monthrange(y,month)[1]
        month = month + 1
#     data['ddd'][i]=month
#     data['ddd'][i]=datetime.datetime(yy, month, jd, int(sec[0]),int(sec[1]),123)
                                
    

ValueError: second must be in 0..59

In [217]:
data.head()

,date,value,da,ddd
0,2015-085-01:47:10.114,19.311741,38.623482,2015-03-26 01:47:00
1,2015-085-01:47:10.882,19.311700,38.623400,2015-03-26 01:47:00
2,2015-085-01:48:22.882,19.311700,38.623400,2015-03-26 01:48:00
3,2015-085-01:49:18.114,19.311741,38.623482,2015-03-26 01:49:00
4,2015-085-01:50:46.882,19.311700,38.623400,2015-03-26 01:50:00


In [224]:
aa=datetime.datetime(2016, 3, 22, 12, 33, 45, 756500)
aa

datetime.datetime(2016, 3, 22, 12, 33, 45, 756500)

In [268]:
from datetime import datetime

datetime_str = '09/19/18 13:55:26'

datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')

print(type(datetime_object))
print(datetime_object)

<class 'datetime.datetime'>
2018-09-19 13:55:26


In [309]:
import datetime
 
# YYYY-MM-DD HH:MM:SS.MS 
xt = datetime.datetime(2018, 10, 10, 3, 38, 1, 112*1000)
print(xt)
 
print(xt.strftime("%a"))

2018-10-10 03:38:01.112000
Wed


In [293]:
a="45.345"
qa=a[-3:]
qa

'345'